# Dick Pooper - the infamous A.I. Rat Race commentator
Based on ChatGPT API and Elevenlabs TTS

Please go to `race_data.json` file and fill in any details you wish about the
 race, the contenders and what not. The structure of the JSON file can be
 freely changed, new keys or datatypes can be added. ChatGPT will read it and
  use these info in his commentary..

Here we import some basic stuff - the API libs mainly.

In [1]:
import os
from pprint import pprint
from time import sleep

import openai
from dotenv import find_dotenv, load_dotenv
from elevenlabs import generate, stream, voices, set_api_key

from prompts import prompts

You need to put your API credentials for Elevenlabs and OpenAI ChatGPT API in
 the `.env` file. Without this the whole thing will either not work or you
 will hit the API rate limits very soon. Please test a little bit. It is hard
  to estimate how many characters for Elevenlabs and how many tokens for
  OpenAI ChatGPT calls will be used in a rat race.

You can get your API_KEYS on the ChatGPT and Elevenlabs websites.

In [2]:
load_dotenv(find_dotenv("./.env"))

True

In [3]:
openai.api_key = os.environ.get("OPENAI_API_KEY")
set_api_key(os.environ.get("ELEVENLABS_API_KEY"))

We WILL get some OpenAI API errors.. This is ambiguosly set to how long we
should sleep after an error.. This should probably be implemented using some
retrying library or something..

In [4]:
WAIT_SEC_ON_API_ERROR = 2

These are the race stages that you can use. This is *the most important part*.

DO NOT RUN INITIAL STAGE MANUALLY - IT WILL BE GENERATED AUTOMATICALLY.

You should always start with an prerace stage before the race start - this
will generate some introductory comments about the contenders, weather or
whatever you put in the `race_data.json` file.

Then you can use the start stage to start a live commentary. The commentator
will announce the start of the race and throw a few jokes or whatnot.

The midrace stage should be run multiple times during the race whenever you
feel like you want to add some commentary.

The finish stage is for announcing the end of the race. It will not give any
details about the winner yet!

When you run the winner stage you should provide the name of the winner after
 space, i.e. `winner Richard`


In [5]:
RACE_STAGES = [
    "initial",
    "prerace",
    "start",
    "midrace",
    "finish",
    "winner",
]

Here comes the code...

In [6]:
def set_tts_voice(voice_name="Arnold", stability=0.1, similarity_boost=0.75):
    voices_list = voices()
    voices_names_list = [voice.name for voice in voices_list]
    chosen_voice_index = voices_names_list.index(voice_name)

    voice = voices()[chosen_voice_index]
    print(voice)
    voice.settings.stability = stability
    voice.settings.similarity_boost = similarity_boost

    return voice

In [7]:
def stream_commentary(prompt, voice):
    audio_stream = generate(
        text=prompt,
        voice=voice,
        stream=True
    )
    return audio_stream

In [8]:
def setup_openai_system_context_message():
    return [
        {
            "role": "system",
            "content": prompts["initial"]
        }
    ]

In [9]:
def call_openai_api(content, chat_history, model="gpt-3.5-turbo", temperature=0.3):
    chat_history.append(
        {
            "role": "user",
            "content": content
        }
    )
    response = openai.ChatCompletion.create(
        model=model,
        messages=chat_history,
        temperature=temperature,
    )

    return response.choices[0].message["content"]

In [10]:
def setup():
    voice = set_tts_voice()
    initial_chat_history = setup_openai_system_context_message()

    return voice, initial_chat_history

In [11]:
def comment(voice, race_stage, chat_history):
    commentary = call_openai_api(prompts[race_stage], chat_history)
    print(f"++++++++++++++++++++++++++++++{race_stage.upper()}++++++++++++++++++++++++++++++")
    print(commentary)
    stream(stream_commentary(commentary, voice))

    return commentary

In [12]:
voice, chat_history = setup()

voice_id='VR6AewLTigWG4xSOukaG' name='Arnold' category='premade' description=None labels={} samples=None settings=VoiceSettings(stability=0.5, similarity_boost=0.75) design=None preview_url='https://storage.googleapis.com/eleven-public-prod/premade/voices/VR6AewLTigWG4xSOukaG/66e83dc2-6543-4897-9283-e028ac5ae4aa.mp3'


This is the main loop that will ask for stage name, run the commentary and
 then ask for another stage name. It will run until you provide the winner
 stage.

In [13]:
while True:
    race_stage = input()
    if race_stage.lower().strip() not in RACE_STAGES:
        print(f"Incorrect race stage. Use one of the following: {', '.join(RACE_STAGES)}")
    if "winner" in race_stage.lower():
        winner = race_stage.split(":")[1].strip()
        comment(voice, " ".join(race_stage, winner))
        break
    try:
        comment_str = comment(voice, race_stage, chat_history)
        dialogue_turn = [
            {
                "role": "user",
                "content": prompts[race_stage],
            },
            {
                "role": "assistant",
                "content": comment_str
            },
        ]
        chat_history.extend(dialogue_turn)
        pprint(chat_history)
    except openai.error.APIConnectionError:
        print("OPENAI API ERROR!!! Moving on!")
        sleep(WAIT_SEC_ON_API_ERROR)


Incorrect race stage. Use one of the following: initial, prerace, start, midrace, finish, winner


KeyError: ''